In [112]:
import requests
import time
import json
import scrapy
import unicodedata
from bs4 import BeautifulSoup
from tqdm import tqdm

In [162]:
# Some global variables
catalog_filename = 'foxnews_catalog.json'
news_filename = 'foxnes_news.json'

### Crawl the Foxnews side for Tesla specific news

In [32]:
url_foxnews = 'https://www.foxnews.com/api/article-search'
query = {
    "isTag" : 'true',
    "searchSelected" : "fox-news/auto/make/tesla",
    "size": 30,
    "offset" : 0
}

catalog = []
for x in tqdm(range(0,1000), ncols=45):
    query['offset'] = x * 30
    r = requests.get(url_foxnews, params=query)
    if r.status_code == 200:
        if r.text != '[]':
            catalog.extend(r.json())
        else:
            print(f"Got blank response at iteration {x*30}")
            break
    time.sleep(0.5)

  1%|██                                                                                                                                               | 14/1000 [00:12<15:10,  1.08it/s]

Got blank response at iteration 420


In [37]:
with open(catalog_filename, 'w') as outfile:
    json.dump(catalog, outfile, indent=4)

### Retrieve the text content from the foxnews catalog

In [81]:
url_foxnews_base = 'https://www.foxnews.com'
with open(catalog_filename, 'r') as infile:
    catalog = json.load(infile)
    
# Ads Words
ad_words = None
with open('ad_words.txt', 'r') as infile:
    ad_words = [line.rstrip('\n') for line in infile.readlines()]

In [160]:
# sanitizer
def SanitizerAdWords(content: str) -> str:
    content = content.lower()
    for a in ad_words:
        content = content.replace(a,'')
    return content

def SanitizerHtml(content: str) -> str:
    return unicodedata.normalize("NFKD",content.rstrip('.').strip()) + '.'

def GetContent(url: str) -> str:
    content = None
    r = requests.get(url)
    if r.status_code == 200:
        html = BeautifulSoup(r.text, 'html.parser')
        article_title = html.find(attrs={"class": "headline"}).text
        article_body = html.find(attrs={"class": "article-body"})    
        content = article_title + '.' + ' '.join([SanitizerHtml(x.text) for x in article_body.find_all('p')])
        content = SanitizerAdWords(content)
        content = unicodedata.normalize("NFKD",content)
    return content

In [161]:
news = []
for inx in tqdm(range(len(catalog))):
    url = url_foxnews_base + catalog[inx]['url']
    content = GetContent(url)
    news.append({'url': url, 'body':content, 'sentiment': ''})   

100%|████████████████████████████████████████████████████████████████████████████████| 409/409 [03:08<00:00,  2.17it/s]


In [163]:
with open(news_filename, 'w') as outfile:
    json.dump(news, outfile, indent=4)

In [158]:
print(news[0]['body'])

tesla's bizarre swipe-to-drive gear selector revealed in video.with a bulletproof body and 500-mile battery-powered range, tesla's electric pickup isn't like every other truck. the first video of tesla's new on-screen "gear" selector system in operation has appeared on twitter. elon musk previously announced the feature, which is debuting on the updated model s and model x vehicles that were revealed in january. twitter user michael hsu posted the clip which shows a driver swiping a car icon in the top left corner of the screen to choose forward and reverse. hsu would not confirm to fox news autos the source of the video, or whether it was a customer or tesla employee demonstrating the system. a separate image posted by @nickhoward shows that neutral is engaged by a separate icon on the screen. drive, reverse and neutral can also be selected using a knob on the steering wheel, but there's apparently a third method that's truly next-level. according to musk, the cars can guess which way

In [96]:
max = 5
for c in catalog:
    print(url_foxnews_base + c['url'])
    if max == 0:
        break
    max -= 1

https://www.foxnews.com/auto/teslas-bizarre-swipe-to-drive-gear-selector
https://www.foxnews.com/auto/tesla-cancels-full-self-driving-expansion-update
https://www.foxnews.com/auto/tesla-autopilot-michigan-state-police-car
https://www.foxnews.com/auto/detroit-police-tesla-autopilot-semi-crash
https://www.foxnews.com/auto/nhtsa-investigating-violent-tesla-crash-semi
https://www.foxnews.com/auto/tesla-fires-self-driving-testers-camera
